In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data[data.isnull()].count()

In [ ]:
X = data.drop('quality', axis=1)
y = data.quality

> ****2. Detect Columns that contains outliers...****

In [ ]:
# For each feature find the data points with extreme high or low values
for feature in X.keys():

    # Q1 (25th percentile of the data) for the given feature
    Q1 = np.percentile(X[feature], q=25)
 
    # Q3 (75th percentile of the data) for the given feature
    Q3 = np.percentile(X[feature], q=75)
 
    # We use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
    interquartile_range = Q3 - Q1
    step = 1.5 * interquartile_range
 
    # Display the outliers
    print("Data points considered outliers for the feature '{}':".format(feature))
    display(X[~((X[feature] >= Q1 - step) & (X[feature] <= Q3 + step))])
 

**What kind of a ML problem is this?If Regression,then can this also be solved as a classification problem?**

This is a supervised learning problem and it can be solved in both regression and classification method, though in classification we will try to minimize the range label to few encoded lables for better correlation and accuracy.We will see both the implementation below after a while.

****EXPLORATORY DATA ANALYSIS****

In [ ]:
X.describe()

In [ ]:
sns.pairplot(X)

**From the above scatterplot we can get some interesting details. For some of the features, the distribution appears to be fairly linear. For some others, the distribution appears to be negatively skewed. So this confirms our initial suspicions — there are indeed some interesting co-dependencies(relying on other features) between some of the features.**

In [ ]:
sns.distplot(y)

In [ ]:
### we can plot heat map to examin the correlation
correlation = X.corr()
# display(correlation)
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(correlation, annot=True, linewidths=0, vmin=-1, cmap="RdBu_r")

**The squares with positive values show direct co-relationships between features. The higher the values, the stronger these relationships are — they’ll be more reddish. That means, if one feature increases, the other one also tends to increase, and vice-versa.**


**The squares that have negative values show an inverse co-relationship. The more negative these values get, the more inversely proportional they are, and they’ll be more blue. This means that if the value of one feature is higher, the value of the other one gets lower.**

**Finally, squares close to zero indicate almost no co-dependency between those sets of features.**

**Fixed Acidity vs. Citric Acid**

In [ ]:
fixedAcidity_citricAcid = X[['citric acid', 'fixed acidity']]
g = sns.JointGrid(x="fixed acidity", y="citric acid", data=fixedAcidity_citricAcid, size=10)
g = g.plot_joint(sns.regplot, scatter_kws={"s": 10})
g = g.plot_marginals(sns.distplot)

**As the citric acid increases the fixed acid also increases linearly.**

In [ ]:

# A new dataframe containing only pH and fixed acidity columns to visualize their co-relations
fixedAcidity_pH = X[['pH', 'fixed acidity']]
gridA = sns.JointGrid(x="fixed acidity", y="pH", data=fixedAcidity_pH, size=10)
#Regression plot in the grid 
gridA = gridA.plot_joint(sns.regplot, scatter_kws={"s": 10})
#Distribution plot in the same grid
gridA = gridA.plot_marginals(sns.distplot)

**Fixed acidity levels increase, the pH levels.A lower pH level is, after all, an indicator of high acidity.**

In [ ]:
volatileAcidity_quality=data[['volatile acidity','quality']]
fig, axs = plt.subplots(ncols=1,figsize=(15,10))
sns.barplot(x='quality', y='volatile acidity', data=volatileAcidity_quality)
plt.title('quality VS volatile acidity')
plt.show()

**We can clearly see that lower the volatile acidity better is the quality of the wine**

In [ ]:
alcohol_quality=data[['alcohol','quality']]
fig, axs = plt.subplots(ncols=1,figsize=(15,10))
sns.barplot(x='quality', y='alcohol', data=alcohol_quality)
plt.title('quality VS alcohol content')
plt.show()

**More the alcohol content better is the quality of wine.**

****DATA PRE-PROCESSING****

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [ ]:
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

**BUILDING  MODEL AND OPTIMIZATION**

**Regression model**

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, pred)*100)
print('MSE:', metrics.mean_squared_error(y_test, pred)*100)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred))*100)

**Classification method**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
clf.fit(X_train,y_train)

In [ ]:
pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score

In [ ]:
print("accuracy:",(accuracy_score(y_test,pred)*100))
